In [3]:
import pandas as pd
import numpy as np
import matplotlib as pl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as pimg
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image
import keras
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import f1_score,classification_report
import os
from tqdm import tqdm


In [4]:
img_size = ()
def load_data():
    datasets = ['src']
    output = []
    
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        for folder in os.listdir(dataset):
            label = folder
            folder_path = os.path.join(dataset, folder)
            for file in tqdm(os.listdir(folder_path)):
                
                img_path = os.path.join(folder_path, file)
                img = cv2.imread(img_path)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # img = cv2.resize(img, IMAGE_SIZE) 
                
                images.append(img)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [5]:
np.random.seed(3)

# train_datagen = ImageDataGenerator(rescale=1./255,
                                #   rotation_range=15,
                                #   width_shift_range=0.1,
                                #   height_shift_range=0.1,
                                #   shear_range=0.5,
                                #   zoom_range=[0.8, 2.0],
                                #   horizontal_flip=True,
                                #   vertical_flip=True,
                                #   fill_mode='nearest'
#                                   )
# test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# train_generator = train_datagen.flow_from_directory('./dataset/training_set',
#                                                     target_size=(500, 500),
#                                                     batch_size=3,
#                                                     class_mode='categorical'
#                                                     )

# test_generator = test_datagen.flow_from_directory('./dataset/test_set',
#                                                   target_size=(500, 500),
#                                                   batch_size=3,
#                                                   class_mode='categorical'
#                                                   )

In [7]:
img_size = (256,256)
batch_size = 16
padding = 'same'
classes = ['climaxForest', 'herbaceous', 'shrub', 'wasteland']
data_path = 'src'

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_path, # same directory as training data
    target_size=img_size,
    batch_size=batch_size,
    subset='validation')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3),padding=padding, input_shape = (*input_size, 3))) 
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3),padding=padding)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3),padding=padding))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3),padding=padding)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model2 = Sequential()
model2.add(Conv2D(16, (3, 3),padding=padding, input_shape = (*img_size, 3))) 
model2.add(Activation('relu'))
model2.add(MaxPooling2D(2,2))

model2.add(Conv2D(32, (3, 3),padding=padding)) 
model2.add(Activation('relu'))
model2.add(MaxPooling2D(2,2))

model2.add(Conv2D(32, (3, 3),padding=padding)) 
model2.add(Activation('relu'))
model2.add(MaxPooling2D(2,2))


model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(len(classes), activation='softmax'))